In [4]:
import pandas as pd

df = pd.read_csv("clean_df.csv")

#Esta es una de las primeras versiones de la función para calcular la disonancia ponderada y el ranking promedio
#Aquí tomo dos canciones de referencia y calculo las diferencias ponderadas con respecto a todas las demás canciones del ds
# Luego simplemente hay que unir los rankings de las dos canciones de referencia y ordenar por el ranking promedio
# Función para calcular la disonancia ponderada
#Para tampoco enrollarme mucho, básicamente, para cada característica (columna) calculo la diferencia entre el valor de referencia
#y el valor de la cancióna evaluar. Esas diferencias se multiplican por un peso que se le asigna a cada característica y se suman
#Ese valor es la disonancia total
#Se hace esto para cada canción y luego se ordena el dataset por disonancia total

def calcular_disonancia(row, referencia, importancias):
    disonancia_total = 0
    for columna, valor_referencia in referencia.items():
        importancia = importancias.get(columna, 1)
        diferencia = abs(row[columna] - valor_referencia)
        disonancia_total += diferencia * importancia
    return disonancia_total

# Función para obtener rankings de disonancia
def obtener_ranking(df, referencia, importancias, nombre_columna_ranking):
    df[nombre_columna_ranking] = df.apply(lambda row: calcular_disonancia(row, referencia, importancias), axis=1)
    df.sort_values(by=nombre_columna_ranking, ascending=True, inplace=True)
    df[nombre_columna_ranking + '_rank'] = range(1, len(df) + 1)
    return df

# Función principal para encontrar la canción con mejor ranking promedio
def encontrar_mejor_ranking_promedio(df, cancion_a, cancion_b, importancias, nombre_columna_cancion, nombre_columna_artista):
    # Obtener referencias de las dos canciones que pongamos
    referencia_a = df.loc[df[nombre_columna_cancion] == cancion_a].iloc[0]
    referencia_b = df.loc[df[nombre_columna_cancion] == cancion_b].iloc[0]

    # Extraer solo las características numéricas relevantes
    adn_a = referencia_a[importancias.keys()].to_dict()
    adn_b = referencia_b[importancias.keys()].to_dict()

    # Obtener rankings para cada referencia
    df = obtener_ranking(df, adn_a, importancias, "disonancia_a")
    df = obtener_ranking(df, adn_b, importancias, "disonancia_b")

    # Calcular el ranking promedio
    df["ranking_promedio"] = (df["disonancia_a_rank"] + df["disonancia_b_rank"]) / 2

    # Ordenar por ranking promedio
    columnas_a_mostrar = [nombre_columna_cancion, nombre_columna_artista, "disonancia_a_rank", "disonancia_b_rank", "ranking_promedio"]
    resultado = df[columnas_a_mostrar].sort_values(by="ranking_promedio")

    return resultado

# Diccionario de importancias
importancias = {
    "male": 1.0,
    "danceable": 0.9,
    "tonal": 0.05,
    "timbre_bright": 0.05,
    "instrumental": 0.9,
    "mood_acoustic": 0.7,
    "mood_aggressive": 1.0,
    "mood_electronic": 0.95,
    "mood_happy": 0.05,
    "mood_party": 1.0,
    "mood_relaxed": 1.0,
    "mood_sad": 1.0

}

# Especificar las dos canciones de referencia
cancion_a = "without me"  
cancion_b = "numb"


# Nombres de las columnas relevantes
nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

# Encontrar la canción con mejor ranking promedio
df_resultado = encontrar_mejor_ranking_promedio(df, cancion_a, cancion_b, importancias, nombre_columna_cancion, nombre_columna_artista)

# Mostrar los resultados
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

print(df_resultado)

FileNotFoundError: [Errno 2] No such file or directory: 'clean_df.csv'

In [5]:
import pandas as pd


df = pd.read_csv("completed_cleaned.csv")

#Este es el que seguramente acabaremos usando. Te devuelve las canciones más cercanas según los parámetros que le indiques y según su relevancia
#En su momento lo puse para el top 5 (y además te incluye la propia canción que le das como referencia). Se puede cambiar para que no saque la primera
#Y también para que saque la peor de todas, la que sería la antítesis de la canción original

# Función para calcular la disonancia ponderada
def calcular_disonancia(row, referencia, importancias):
    disonancia_total = 0
    for columna, valor_referencia in referencia.items():
        importancia = importancias.get(columna, 1)
        diferencia = abs(row[columna] - valor_referencia)
        disonancia_total += diferencia * importancia
    return disonancia_total

# Función para obtener rankings de disonancia
def obtener_ranking(df, referencia, importancias, nombre_columna_ranking):
    df[nombre_columna_ranking] = df.apply(lambda row: calcular_disonancia(row, referencia, importancias), axis=1)
    df.sort_values(by=nombre_columna_ranking, ascending=True, inplace=True)
    df[nombre_columna_ranking + '_rank'] = range(1, len(df) + 1)
    return df

# Función para obtener el top 5 de canciones menos disonantes para cada referencia
def obtener_top_5_por_referencia(df, canciones_referencia, importancias, nombre_columna_cancion, nombre_columna_artista):
    resultados_top_5 = {}

    for idx, cancion in enumerate(canciones_referencia, start=1):
        # Obtener la referencia de la canción actual
        referencia = df.loc[df[nombre_columna_cancion] == cancion].iloc[0]
        adn = referencia[importancias.keys()].to_dict()

        # Obtener ranking para la canción actual
        df_ranking = obtener_ranking(df.copy(), adn, importancias, f"disonancia_{idx}")

        # Seleccionar el top 5
        top_5 = df_ranking[[nombre_columna_cancion, nombre_columna_artista, f"disonancia_{idx}"]].head(5)
        resultados_top_5[cancion] = top_5

    return resultados_top_5

# Diccionario de importancias
importancias = {
    "male": 0.05,
    "danceable": 0.05,
    "tonal": 0.05,
    "timbre_bright": 0.05,
    "instrumental": 0.05,
    "mood_acoustic": 0.05,
    "mood_aggressive": 0.05,
    "mood_electronic": 0.05,
    "mood_happy": 0.05,
    "mood_party": 1.0,
    "mood_relaxed": 0.05,
    "mood_sad": 0.05
}

# Lista de canciones de referencia
canciones_referencia = ["ave maria", "satisfaction", "little blue"]  # Añade más canciones si lo deseas

# Nombres de las columnas relevantes
nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

# Obtener el top 5 de canciones menos disonantes para cada referencia
resultados_top_5 = obtener_top_5_por_referencia(df, canciones_referencia, importancias, nombre_columna_cancion, nombre_columna_artista)

# Mostrar los resultados
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

for cancion, top_5 in resultados_top_5.items():
    print(f"\nTop 5 canciones menos disonantes para '{cancion}':")
    print(top_5)

FileNotFoundError: [Errno 2] No such file or directory: 'completed_cleaned.csv'

In [ ]:
import pandas as pd

df = pd.read_csv("clean_df.csv")

# Este otro código es para calcular la canción "central". Para resolver el problema del arranque en frío.
#Lo que hace es comparar todas con todas (por pares de puntos) y calcular la diferencia entre ellas. Esto es a lo que me refería con "distancia"
#La idea es que las canciones que están a menor distancia de todas las demás es porque están más centralizadas

#Si visualizamos el dataset como un espacio n-dimensional, tenemos que cada canción es un punto en ese espacio. Por simplificar, digamos que está en 2d
#Si quisiéramos encontrar el punto más central, lo que tenemos que hacer es encontrar el punto que está "menos lejos" de todos los demás
#Podemos medir lo lejos que está un punto de otro (y aquí sí estamos usando distancia euclídea) y hacer eso para todos los puntos
#(Paréntesis. No será esto de comparar todos los pares de puntos demasiado costoso teniendo en cuenta la cantidad de canciones que manejamos? No. El 
#motivo es que, como podemos calcular, la cantidad de comparaciones de pares de puntos es simplemente con n*(n+1)/2 donde n es la cantidad de puntos. 
#Si a esto le sumamos que nunca vamos a tener más de 50 canciones del usuario a comparar, vemos que no es un problema, ya que nunca estamos pasando de 
# 50*51/2 = 1275 comparaciones)
#Ahora que ya sabemos que el algoritmo para encontrar la canción central funciona, voy a pasar a explicar por qué importa esto

#No sería más interesante encontrar la media de todas las canciones para hacer algo que satisfaga mejor al usuario? No tendrá más sentido 
#contemplar todo lo que sabemos en lugar de simplemente una canción?
#Estas dos preguntas llevaron a bastantes códigos como el que dejo aquí:

import pandas as pd

df = pd.read_csv("clean_df.csv")


def calcular_disonancia(row, referencia, importancias):
    disonancia_total = 0
    for columna, valor_referencia in referencia.items():
        importancia = importancias.get(columna, 1)
        diferencia = abs(row[columna] - valor_referencia)
        disonancia_total += diferencia * importancia
    return disonancia_total

#Aquí se calculan las diferencias entre las canciones de referencia y las del dataset
def obtener_ranking(df, referencia, importancias, nombre_columna_ranking):
    df[nombre_columna_ranking] = df.apply(lambda row: calcular_disonancia(row, referencia, importancias), axis=1)
    df.sort_values(by=nombre_columna_ranking, ascending=True, inplace=True)
    df[nombre_columna_ranking + '_rank'] = range(1, len(df) + 1)
    return df

# Función principal para encontrar la canción con mejor ranking promedio
def encontrar_mejor_ranking_promedio(df, cancion_a, cancion_b, importancias, nombre_columna_cancion, nombre_columna_artista):
    #Obtener referencias de las dos canciones que pongamos
    referencia_a = df.loc[df[nombre_columna_cancion] == cancion_a].iloc[0]
    referencia_b = df.loc[df[nombre_columna_cancion] == cancion_b].iloc[0]

    #Las ponderamos por importancia
    adn_a = referencia_a[importancias.keys()].to_dict()
    adn_b = referencia_b[importancias.keys()].to_dict()

    #SObre las ponderaciones sacamos el ranking
    df = obtener_ranking(df, adn_a, importancias, "disonancia_a")
    df = obtener_ranking(df, adn_b, importancias, "disonancia_b")

    #El promedio es simplemente:
    df["ranking_promedio"] = (df["disonancia_a_rank"] + df["disonancia_b_rank"]) / 2

    #Y las ordenamos
    columnas_a_mostrar = [nombre_columna_cancion, nombre_columna_artista, "disonancia_a_rank", "disonancia_b_rank", "ranking_promedio"]
    resultado = df[columnas_a_mostrar].sort_values(by="ranking_promedio")

    return resultado

#Este es el diccionario de importancias que puede emplear el usuario
importancias = {
    "male": 1.0,
    "danceable": 0.9,
    "tonal": 0.05,
    "timbre_bright": 0.05,
    "instrumental": 0.9,
    "mood_acoustic": 0.7,
    "mood_aggressive": 1.0,
    "mood_electronic": 0.95,
    "mood_happy": 0.05,
    "mood_party": 1.0,
    "mood_relaxed": 1.0,
    "mood_sad": 1.0

}

#Canciones de referencia
cancion_a = "without me"  
cancion_b = "numb"


nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

df_resultado = encontrar_mejor_ranking_promedio(df, cancion_a, cancion_b, importancias, nombre_columna_cancion, nombre_columna_artista)


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

print(df_resultado)
#(Esta es una de las primeras versiones de la función para calcular la disonancia ponderada y el ranking promedio
#Aquí tomo dos canciones de referencia y calculo las diferencias ponderadas con respecto a todas las demás canciones del ds
# Luego simplemente hay que unir los rankings de las dos canciones de referencia y ordenar por el ranking promedio
# Función para calcular la disonancia ponderada
#Para cada característica (columna) calculo la diferencia entre el valor de referencia y el valor de la cancióna evaluar. 
#Esas diferencias se multiplican por un peso que se le asigna a cada característica y se suman
#Ese valor es la disonancia total
#Se hace esto para cada canción y luego se ordena el dataset por disonancia total)

#En primer lugar, estamos contemplando lo que sabemos para elegir específicamente esta canción, por lo que no estamos sobresimplificando
#En segundo lugar, voy a poner un ejemplo muy gráfico de por qué combinar dos canciones y usar esos parámetros promedios para recomendar no funciona
#Supongamos que hay un finlandés que suele hacer su ritual de salir de una sauna y meterse al agua helada varias veces. Sería un error decir "Voy a 
#combinar sus gustos de temperaturas y voy a recomendarle que se meta en agua templada, seguro que le encanta"
#Lo mismo ocurre con las canciones. Si un usuario tiene playlists con estilos marcados pero que combinan bien entre sí, no podemos mezclar esos dos
#estilos y recomendarle una fusión, porque lo que está buscando es una cosa u otra.


#Por qué utilizar canciones parecidas en lugar de canciones complementarias?
#Este es un approach interesante pero muy difícil de implementar. Las dos opciones a considerar fueron:
#      - Entrenar un modelo con 5M de playlists para que aprenda qué canciones suelen ir en la misma playlist (Muy lento y costoso)
#      - Utilizar información de usuarios parecidos para recomendar canciones (No hay forma de acceder a información de usuarios por temas de privacidad)

#Es por esto que la única opción disponible, práctica y realista era recomendar en base a la similitud de una canción a otra

#_________________________________________________________________________________________________________________________________________________________

# Función para calcular la tasa de diferencia entre las canciones de referencia
def calcular_tasa_diferencia_referencias(df, referencias, columnas_parametros):
    diferencias = []
    canciones_referencia = []
    promedio_diferencias = {}
    
    # Extraer las canciones de referencia del dataframe
    for referencia in referencias:
        cancion_df = df[(df["song_name"] == referencia["cancion"]) & (df["artist_name"] == referencia["artista"])]
        if not cancion_df.empty:
            canciones_referencia.append(cancion_df.iloc[0])
            promedio_diferencias[referencia["cancion"]] = 0
    
    # Comparar todas las canciones de referencia entre sí
    for i in range(len(canciones_referencia)):
        for j in range(i + 1, len(canciones_referencia)):
            song_1 = canciones_referencia[i]
            song_2 = canciones_referencia[j]
            diferencia_total = 0
            
            for columna in columnas_parametros:
                valor_1 = song_1[columna] * 10
                valor_2 = song_2[columna] * 10
                diferencia_total += (abs(valor_1 - valor_2)) ** 2
            
            diferencias.append({
                "song_1": song_1["song_name"],
                "artist_1": song_1["artist_name"],
                "song_2": song_2["song_name"],
                "artist_2": song_2["artist_name"],
                "tasa_diferencia": diferencia_total
            })
            
            # Acumular diferencias para el promedio
            promedio_diferencias[song_1["song_name"]] += diferencia_total
            promedio_diferencias[song_2["song_name"]] += diferencia_total
    
    # Calcular el promedio de diferencia para cada canción
    for cancion in promedio_diferencias:
        promedio_diferencias[cancion] /= (len(canciones_referencia) - 1)
    
    # Convertir a DataFrame y ordenar
    diferencias_df = pd.DataFrame(diferencias)
    diferencias_df.sort_values(by="tasa_diferencia", ascending=True, inplace=True)
    
    # Crear DataFrame con promedios y ordenarlo
    promedio_df = pd.DataFrame(list(promedio_diferencias.items()), columns=["song_name", "promedio_diferencia"])
    promedio_df.sort_values(by="promedio_diferencia", ascending=True, inplace=True)
    
    return diferencias_df, promedio_df

# Diccionario de importancias
importancias = {
    "male": 0.5,
    "danceable": 1.2,
    "tonal": 0.8,
    "timbre_bright": 1.0,
    "instrumental": 1.5,
    "mood_acoustic": 0.7,
    "mood_aggressive": 1.3,
    "mood_electronic": 1.1,
    "mood_happy": 0.9,
    "mood_party": 1.4,
    "mood_relaxed": 0.6,
    "mood_sad": 1.0
}

# Lista de canciones de referencia con artista
referencias = [
    {"cancion": "mas guapa que cualquiera", "artista": "joaquin sabina"},
    {"cancion": "contrabando", "artista": "joaquin sabina"},
    {"cancion": "mi primo el nano", "artista": "joaquin sabina"},
    {"cancion": "while were young", "artista": "barry manilow"}
]

# Nombres de las columnas relevantes
nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

# Obtener la tasa de diferencia entre las canciones de referencia
columnas_parametros = list(importancias.keys())
diferencias_df, promedio_df = calcular_tasa_diferencia_referencias(df, referencias, columnas_parametros)

# Mostrar los resultados
print("\nTasa de diferencia entre canciones de referencia:")
print(diferencias_df)

print("\nRanking de canciones por menor diferencia promedio:")
print(promedio_df)


In [1]:
#En este apartado voy a explicar el funcionamiento de los algoritmos utilizados en el recomendador final, así como por qué han sido
#estos los elegidos frente a otros:

#K-Nearest Neighbors (KNN)
#En primer lugar, como he comentado en otros apartados, la idea de medir la distancia de unas canciones a otras en base a KNN tiene sentido, pero
#también parece sobresimplificar el problema.
#En un escenario en el que solamente tengamos etiquetas con las que comparar, KNN sería una opción muy buena, pero en este caso contamos con 
#información extra. 
#Voy a mostrarlo:

#En un sistema de etiquetas, si una canción tiene als etiquetas "bailable" y "tonal", entonces podremos pensar que la presencia o ausencia de estas
#etiquetas será lo que se pueda emplear para determinar cómo de parecidas son dos canciones entre sí
#Sin embargo, en las bases de datos que manejamos tenemos algo mucho mejor: el grado en que aparecen cada una de estas características
#Esto es una gran ventaja, porque es mucho más razonable hablar de una canción "muy bailable" o "poco bailable" en lugar de simplemente "bailable"

#Esto ocurre con todas las características.
#Hace esto que KNN sea inútil? No, pero colapsar datos como 0,57 o 0,8 y convertir ambos a 1 (y hacer lo mismo con 0,42 y 0,002 ---> 0) es un desperdicio
#de información.

#Una vez aclarado por qué KNN no nos pareció suficiente, paso a explicar cómo llegué a la versión "refinada" del KNN:

#Antes de nada, introduzco aquí el concepto de "disonancia". Si tenemos dos valores para "bailable", por ejemplo 0,8 y 0,2, entonces la disonancia 
#será simplemente la diferencia entre esas dos características (0,6 en este caso).
#Si tenemos más características, entonces la diferencia total será la suma de todas ellas. De esta manera podemos comparar todas las canciones del ds
#con nuestra canción de referencia.
#(De nuevo, alguien puede preguntarse qué pasa si dos canciones tienen estos dos grupos de *diferencias*, es decir, no de valores:
#     0,3 y 0,6 vs 0,45 y 0,45. 
# En este caso la disonancia total será la misma, sin embargo es probable que el usuario siga prefiriendo una sobre otra, y el algoritmo actual no tiene
# forma de contemplar esto)
#Bien, pues aquí es donde pasamos a la segunda parte del algoritmo

#diccionario de características de ejemplo = {
    #"male": 0.5,
    #"danceable": 0.22,
    #"tonal": 0.8,
    #"timbre_bright": 0.83,
    #"instrumental": 0.34,
    #"mood_acoustic": 0.7,
    #"mood_aggressive": 1.0,
    #"mood_electronic": 0.02,
    #"mood_happy": 0.912,
    #"mood_party": 1.0,
    #"mood_relaxed": 0.05,
    #"mood_sad": 0.088
#}

#Si miramos este diccionario y pensamos en qué características queremos introducir para que nos devuelva las canciones ordenadas por nuestro gusto, 
#seguramente haya alguna característica que sea la que más nos interesa. (Es importante aclarar que la que más nos interesa no tiene por qué ser la más
#extrema, es decir, podemos querer que nuestra canción sea un 66% electronic y que esta cantidad sea muy importante para nostros). Por las mismas, habrá
#características que nos sean indiferentes en alguna búsqueda, por lo que sería contraproducente que el algoritmo tratase de ajustarse lo mejor posible
#a valores que nos dan igual.
#Aquí es donde entra el segundo factor: ponderar las características.
#Una característica que es muy susceptible de ser ponderada alta es "mood_party" ---> Si estamos buscando hacer una playlist de fiesta pondemos:
#                                                                   "mood_party": 100%
#                                                                   "importancia de se le da a la cantidad de mood_party": 100%
#Por el contrario, si estamos buscando una playlist de relajación pondremos:
#                                                                   "mood_relaxed": 100%
#                                                                   "mood_party": 0%
# Y le daremos mucha importancia a las dos, porque queremos que se respeten al máximo esas cantidades

#Dejo por aquí el algoritmo final:


#_________________________________________________________________________________________________________________________________________________________

#Ventajas de este algoritmo:
# - No sobresimplifica el problema ---> Estamos jugando con todos los datos que tenemos de una manera que tiene sentido
# - Puede ser muy preciso si se ajusta bien (el usuario puede tener curva de aprendizaje)
# - Es muy rápido 
# - No necesita reentrenamiento ---> A lo largo del proyecto hemos cambiado, añadido, modificado, eliminado, ajustado y hecho de todo con las bases de datos
#Con lo cual tener un algoritmo que sea capaz de tomar cualquier cantidad de datos en cualquier momento es una ventaja a la hora de evaluar
# - Es interactivo ---> A muchos músicos, aficionados a la música etc nos gusta tener una especie de "mesa de mezclas" con la que jugar
# - Está hecho por personas, con lo cual todas las decisiones y características tienen sentido y obedecen a una idea (no es un algoritmo
#que de manera misteriosa devuelve outcomes interesantes pero a veces incomprensibles y que solamente se pueden cambiar usando otro modelo o reentrenando)
#Se puede combinar con facilidad con otros algoritmos de filtrado. Por ejemplo, para hacer que no te recomiende ciertas canciones, para matizar otros
#modelos más simples o para complementar modelos de recomendación complejos pero de otros tipos (como el recomendador basado en contexto)

#Desventajas:
# - No es muy rápido ---> En las pruebas con un ds de 130k canciones y 12 características por canción estaba tardando unos 10-12 segundos en devolver 
#las recomendaciones
# - Depende de la calidad de los datos ---> Esto no es nuevo, al final como analistas de datos siempre estamos dependiendo de la calidad de los datos. Por
#poner un poco más de contexto, hay canciones con un grado de femineidad del 63% que están cantadas por un hombre, lo cual no es estrictamente incorrecto
#y podemos entender por qué el algoritmo que le asignaba los valores lo ha hecho así, pero es contraintuitivo para mucha gente, y por tanto para el usuario
# - Las métricas son difíciles de implementar ---> No son tan directas por estar tratando con arte. Aun así, tenemos. Están explicadas en el apartado de métricas

#Comentarios:
#La base de datos es relativamente pequeña (130k canciones), pero es muchísimo más grande que la cantidad de canciones que una persona puede tener
#en su registro de canciones que conoce. Dado que como usuario no conoces la gran mayoría de esas 130k canciones del ds, es muy probable que las 
#recomendaciones que saque sean nuevas para ti. 
#Los tres miembros del equipo hemos descubierto canciones que nos han gustado mucho, algunas muy poco conocidas, gracias a este algoritmo
#No todo el mundo está dispuesto a hacer el esfuerzo de descubrir canciones nuevas. Los usuarios que ya saben qué canción quieren escuchar no necesitan 
#este algoritmo, por lo que el objetivo principal, que es el de ofrecerle canciones relevantes al usuario, no se ve contrapesado por el hecho de que en 
#muchas ocasiones las canciones recomendadas le serán desconocidas hasta ese momento" 